In [1]:
import numpy as np
import os, sys, torch
sys.path.append("src")
from src.model import ShortChunkCNN_Res

wavs_path = "/data/avramidi/harmonix/audio_wav/"
wavs_path = "/home/kavra/Datasets/DEAP/stimuli_wav/"
song_file = os.listdir(wavs_path)[1]

In [ ]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"
input_length = 73600
batch_size = 32
model = ShortChunkCNN_Res()
model.to(device)

# MSD model is not compatible !
model_load_path = "data/pretrained_models/focal_loss_all_data/"

state = torch.load(os.path.join(model_load_path, "best_model.pth"), map_location=device)
if "spec.mel_scale.fb" in state.keys():
    model.spec.mel_scale.fb = state["spec.mel_scale.fb"]
model.load_state_dict(state)

In [9]:
model.eval()
eg_input = torch.randn(batch_size, 73600)
# outputs 56 classes and 512D embeds
emb = model(eg_input.to(device))

In [ ]:
npy_path = os.path.join("../data/npy_audios", song_file.replace(".wav", ".npy"))
raw = np.load(npy_path, mmap_mode="r")
hop = (len(raw) - input_length) // batch_size  # split chunk
input = [
    torch.Tensor(raw[i * hop : i * hop + input_length]).unsqueeze(0)
    for i in range(batch_size)
]
out = model(input.to(device))